<a href="https://colab.research.google.com/github/subhascodes/Responsible_AI_Class/blob/main/Responsible_AI_(Graph_Models_and_Similarity).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

df = pd.read_csv('/movielens_sample_dataset_1500.csv')  # Update with your filename
print(df.head())
print(df.columns)


   userId  movieId  rating   timestamp                               title  \
0      18        1     3.0  1108273483                    Toy Story (1995)   
1      18        3     1.5  1108273290             Grumpier Old Men (1995)   
2      18        5     4.0  1108273945  Father of the Bride Part II (1995)   
3      18        7     4.0  1108273895                      Sabrina (1995)   
4      18       10     4.0  1108273566                    GoldenEye (1995)   

                                        genres  year  
0  Adventure|Animation|Children|Comedy|Fantasy  1995  
1                               Comedy|Romance  1995  
2                                       Comedy  1995  
3                               Comedy|Romance  1995  
4                    Action|Adventure|Thriller  1995  
Index(['userId', 'movieId', 'rating', 'timestamp', 'title', 'genres', 'year'], dtype='object')


In [ ]:
import pandas as pd
import networkx as nx

# Load dataset
df = pd.read_csv('/content/movielens_sample_dataset_1500.csv')  # Corrected filename

G = nx.Graph()

# Add Movie nodes with attributes
for idx, row in df.iterrows():
    movie_node = f"Movie_{row['movieId']}"
    # Add movie node with title, year
    if not G.has_node(movie_node):
        G.add_node(movie_node, type='movie', title=row['title'], year=row['year'])

    # Add Genre nodes and edges
    if pd.notna(row['genres']):
        genres = row['genres'].split('|')
        for genre in genres:
            if not G.has_node(genre):
                G.add_node(genre, type='genre')
            # Add edge between movie and genre
            if not G.has_edge(movie_node, genre):
                G.add_edge(movie_node, genre, relation='has_genre')

# (Optional) Add edges between movies sharing genres to model similarity explicitly
movies = [n for n, attr in G.nodes(data=True) if attr.get('type') == 'movie']
for i in range(len(movies)):
    for j in range(i+1, len(movies)):
        movie1 = movies[i]
        movie2 = movies[j]
        genres1 = {nbr for nbr in G.neighbors(movie1) if G.nodes[nbr]['type'] == 'genre'}
        genres2 = {nbr for nbr in G.neighbors(movie2) if G.nodes[nbr]['type'] == 'genre'}
        # If they share at least one genre, create similarity edge
        if genres1 & genres2:
            G.add_edge(movie1, movie2, relation='similar_genre')

# Now graph G has movies connected to genres and movies connected to similar movies by genre